In [1]:
# %cd /content/drive/MyDrive/LLM_fintune/

/content/drive/MyDrive/LLM_fintune


In [3]:
# !pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [2]:
from huggingface_hub import login
login()

In [5]:
!pip install torch

# 新增區段

In [3]:
import tqdm
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer
import gc

In [4]:
#Force garbage collection
gc.collect()
def display_cuda_memory():
    print("\n--------------------------------------------------\n")
    print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
    print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))
    print("\n--------------------------------------------------\n")

display_cuda_memory()


--------------------------------------------------

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB

--------------------------------------------------



In [5]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

base_model = "facebook/opt-350m"
# att_json = "/content/drive/MyDrive/LLM_fintune/dataset/Reconnaissance.json"

# language_dataset = load_dataset("fengtc/GuanacoDataset")

att = 'jiandong/crimson-attck-vectors'
new_model = "/content/drive/MyDrive/LLM_fintune/models/opt-att-finetune-350m"

# 4-bit Quantization Configuration
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=compute_dtype, bnb_4bit_use_double_quant=False)

In [6]:
# dataset1 = load_dataset('json', data_files= chinese_json,split="train")
dataset = load_dataset(att,split="train")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/820 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'attck_id', 'attck_name', 'description', 'kill_chain_phases', 'domains', 'tactic_type', 'embedding'],
    num_rows: 820
})


In [ ]:
!pwd

# 新增區段

In [7]:
# Load model with 4-bit precision
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=quant_config, device_map={"": 0})
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

Some weights of OPTForCausalLM were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
print(dataset[0])

In [8]:
def formatting_prompts_func(data):
  output_texts = []
  for i in range(len(data)):
      text = f"""id: {data['id'][i]}\n ,attck_id: {data['attck_id'][i]} , description: {data['description'][i]} ,
        kill_chain_phases: {data['kill_chain_phases'][i]} , domains: {data['domains'][i]} , tactic_type: {data['tactic_type'][i]}
        """
      output_texts.append(text)
  return output_texts


In [9]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [10]:
# Set PEFT Parameters
peft_params = LoraConfig(lora_alpha=8, lora_dropout=0.05, r=64, bias="none", task_type="CAUSAL_LM")

# Define training parameters
training_params = TrainingArguments(
  output_dir=new_model, num_train_epochs=30, per_device_train_batch_size=4, gradient_accumulation_steps=1,
  optim="paged_adamw_32bit", save_steps=10, logging_steps=5, learning_rate=2e-4,
  weight_decay=0.001, fp16=False, bf16=False, max_grad_norm=0.3, max_steps=-1, warmup_ratio=0.03,
  group_by_length=True, lr_scheduler_type="constant", report_to="tensorboard"
)

# Initialize the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    formatting_func=formatting_prompts_func,
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)

# Force clean the pytorch cache
gc.collect()
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/820 [00:00<?, ? examples/s]

In [11]:
# Train the model
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
5,4.425900
10,4.130800
15,3.907300
20,3.607900
25,3.437100
30,3.178500
35,3.061000
40,2.813900
45,2.707600
50,2.543700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=60, training_loss=3.213852135340373, metrics={'train_runtime': 101.8063, 'train_samples_per_second': 2.357, 'train_steps_per_second': 0.589, 'total_flos': 42000639787008.0, 'train_loss': 3.213852135340373, 'epoch': 30.0})

In [12]:
from transformers import pipeline

prompt = "What is Descript of att_id T1430.001"

base = pipeline("text-generation", model=base_model)

print(base(prompt)[0]["generated_text"])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


What is Descript of att_id T1430.001?

The following is a


In [14]:
model.eval()

# finetune = pipeline("text-generation", model=model) # x
# print(finetune(prompt)[0]["generated_text"])


Sample_input_text_tokens = tokenizer(prompt, return_tensors="pt")
Sample_input_text_tokens.to('cuda')
out = model.generate(**Sample_input_text_tokens, max_new_tokens=100)

# decode the generated text
print(tokenizer.decode(out[0]))

</s>What is Descript of att_id T1430.001?

A - Adversaries may use att_id to identify a user, but they may also use it to identify a device, such as a smartphone. If the device is not connected to the network, they may not be able to access the data.

- Adversaries may use att_id to identify a user, but they may also use it to identify a device, such as a smartphone. If the device is not connected to the network, they may not be


In [ ]:
# Save the model and tokenizer
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

In [ ]:
# push to hub

model.push_to_hub("Xcvddax/finetune-att-ck-model")